<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [25]:
import pandas as pd
import psycopg2
import sqlalchemy as sa
import requests
from bs4 import BeautifulSoup
from pprint import pprint

In [263]:
# предлагаемый в курсе способ подключение к БД выдаёт предупреждение о необходимости подключения с помощью интсрументов
# библиотеки SQLAlchemy. Это связано с политекой разработчиков pandas, поэтому курс в этой части вероятно требует
# переработки
connection = f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}'

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [264]:
query_3_1 = f'''select *
           from vacancies
'''

In [266]:
# результат запроса
df_v = pd.read_sql_query(query_3_1, connection)
df_v.shape[0]


49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [30]:
# текст запроса
query_3_2 = f'''select *
           from employers
'''

In [31]:
# результат запроса
df_e = pd.read_sql_query(query_3_2, connection)
df_e.shape[0]

23501

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [32]:
# текст запроса
query_3_3 = f'''select *
           from areas
'''

In [33]:
# результат запроса
df_a = pd.read_sql_query(query_3_3, connection)
df_a.shape[0]

1362

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [34]:
# текст запроса
query_3_4 = f'''select *
           from industries
'''

In [35]:
# результат запроса
df_i = pd.read_sql_query(query_3_4, connection)
df_i.shape[0]

294

***

### выводы по предварительному анализу данных

Выборка содержит около 50 тысяч вакансий в почти 300 сферах деятельности от более 23 тысяч работодателей в 1362 городах 

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [37]:
# текст запроса
query_4_1 = f'''select
           count(v.id) cnt,
           a.name
           from vacancies v
           join 
           areas a
           on a.id = v.area_id
           group by a.id
           order by 1 desc
           limit 5
'''
df_4_1 = pd.read_sql_query(query_4_1, connection)

In [38]:
# результат запроса
df_4_1

,cnt,name
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [39]:
df_v

,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,55312386,Компьютерный Мастер,Пользователь ПК\tРабота в команде\tРемонт ноут...,Полный день,Нет опыта,Полная занятость,64000.0,NaN,1516,5724811
1,55843014,Системный администратор,Средства криптографической защиты информации\t...,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,130,4903713
2,54525964,Lead Java Developer to Poland,Spring Framework\tSQL\tHibernate ORM\tJava\tGit,Удаленная работа,От 3 до 6 лет,Полная занятость,NaN,NaN,160,69961
3,54525965,Lead Java Developer to Poland,Spring Framework\tSQL\tHibernate ORM\tJava\tGit,Удаленная работа,От 3 до 6 лет,Полная занятость,NaN,NaN,159,69961
4,55354053,Специалист службы поддержки с техническими зна...,None,Удаленная работа,Нет опыта,Частичная занятость,15000.0,NaN,1955,1740
...,...,...,...,...,...,...,...,...,...,...
49192,55589514,Специалист по информационным технологиям,Электронный документооборот\t1С: Предприятие 8...,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,66,5109332
49193,55029668,Руководитель процессного офиса,Оптимизация бизнес-процессов\tСтратегическое м...,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,26,51333
49194,66172263,Начальник сервисной службы,Организаторские навыки\tУправление персоналом\...,Полный день,От 1 года до 3 лет,Полная занятость,60000.0,NaN,1930,4094700
49195,66180086,Senior IOS разработчик,Swift\tRealm\tiOS\tАнглийский язык\tMVP\tMVVM\...,Полный день,От 1 года до 3 лет,Полная занятость,146456.0,244094.0,160,4469939


In [40]:
# текст запроса
query_4_2 = f'''select
           salary_from,
           salary_to
           from vacancies v
           where salary_from is not null
           or salary_to is not null
'''

In [41]:
# результат запроса
df_4_2 = pd.read_sql_query(query_4_2, connection)
df_4_2.shape[0]

24073

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [42]:
# текст запроса
query_4_3 = f'''select
           round(avg(salary_from)) "from",
           round(avg(salary_to)) "to"
           from vacancies v
           
'''

In [43]:
# результат запроса
df_4_3 = pd.read_sql_query(query_4_3, connection)
df_4_3

,from,to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [44]:
# текст запроса
query_4_4 = f'''select
           schedule || '-' || employment comb,
           count(id)
           from vacancies v
           group by (schedule || '-' || employment)
           order by 2 desc
           offset 1 limit 1
'''           

In [45]:
# результат запроса
df_4_4 = pd.read_sql_query(query_4_4, connection)
df_4_4

,comb,count
0,Удаленная работа-Полная занятость,7802


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [46]:
# текст запроса
query_4_5 = f'''select
           experience,
           count(id)
           from vacancies v
           group by experience
           having experience is not null
           order by 2
    
'''           

In [47]:
# результат запроса
df_4_5 = pd.read_sql_query(query_4_5, connection)
df_4_5

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [48]:
# выводы по детальному анализу вакансий

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [49]:
df_v.head()

,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,55312386,Компьютерный Мастер,Пользователь ПК\tРабота в команде\tРемонт ноут...,Полный день,Нет опыта,Полная занятость,64000.0,NaN,1516,5724811
1,55843014,Системный администратор,Средства криптографической защиты информации\t...,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,130,4903713
2,54525964,Lead Java Developer to Poland,Spring Framework\tSQL\tHibernate ORM\tJava\tGit,Удаленная работа,От 3 до 6 лет,Полная занятость,NaN,NaN,160,69961
3,54525965,Lead Java Developer to Poland,Spring Framework\tSQL\tHibernate ORM\tJava\tGit,Удаленная работа,От 3 до 6 лет,Полная занятость,NaN,NaN,159,69961
4,55354053,Специалист службы поддержки с техническими зна...,None,Удаленная работа,Нет опыта,Частичная занятость,15000.0,NaN,1955,1740


In [50]:
# текст запроса
query_5_1 = f'''select
           e.name,
           count(v.id)
           from vacancies v
           join
           employers e
           on e.id = v.employer_id
           group by e.name
           order by 2 desc
           limit 5
    
'''       

In [51]:
# результат запроса
df_5_1 = pd.read_sql_query(query_5_1, connection)
df_5_1

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [52]:
# текст запроса
query_5_2 = f'''select
           a.name,
           count(v.id),
           count(e.id)
           from
           areas a
           left join
           vacancies v
           on a.id = v.area_id
           left join 
           employers e
           on a.id =  e.area
           group by a.name
           having count(v.id) = 0
           order by 3 desc
           limit 5
    
'''       

In [53]:
# результат запроса
df_5_2 = pd.read_sql_query(query_5_2, connection)
df_5_2

,name,count,count
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Беларусь,0,18


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [54]:
df_e

,id,name,area
0,2393,"Программный Продукт, ИТ-компания",1
1,72977,БАРС Груп,88
2,3155,"Бест, Торгово-производственная компания, Екате...",3
3,675,КОРУС Консалтинг,2
4,1840010,филиал ФКУ Налог-Сервис ФНС России в Республик...,88
...,...,...,...
23496,1441774,МарКо,88
23497,4718575,MKomov Studio,73
23498,3702770,АКИБ Ипотека-банк,2759
23499,5636092,ГИМ-КЛИНИКА,99


In [55]:
# текст запроса
query_5_3 = f'''select
           e.name,
           count(distinct v.area_id)
           from
           vacancies v
           join
           employers e
           on e.id = v.employer_id
           group by e.id
           order by 2 desc
           limit 5
    
'''       

In [56]:
# результат запроса
df_5_3 = pd.read_sql_query(query_5_3, connection)
df_5_3

,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [57]:
# текст запроса
query_5_4 = f'''select
           count(e.id)
           from
           employers e
           left join
           employers_industries ei
           on e.id = ei.employer_id
           where ei.employer_id is null
           
    
'''       

In [58]:
# результат запроса
df_5_4 = pd.read_sql_query(query_5_4, connection)
df_5_4

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [59]:
# текст запроса
query_5_5 = f'''select
           e.name
           from
           employers e
           join
           employers_industries ei
           on e.id = ei.employer_id
           group by e.name
           having count(ei.industry_id) = 4
           order by 1
           offset 2 limit 1
           
    
'''       

In [60]:
# результат запроса
df_5_5 = pd.read_sql_query(query_5_5, connection)
df_5_5

,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [61]:
# текст запроса
query_5_6 = f'''select
           count(e.name)
           from
           employers e
           join
           employers_industries ei
           on e.id = ei.employer_id
           join
           industries i
           on i.id = ei.industry_id
           where i.name = 'Разработка программного обеспечения'
           
    
'''

In [62]:
# результат запроса
df_5_6 = pd.read_sql_query(query_5_6, connection)
df_5_6

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%%D0%93%%D0%%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [63]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8#%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D0%BE%D0%B2'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
links = page.find_all('a')
cities_bill = [link.text for link in links[91:107]]
cities_bill = tuple(cities_bill)
cities_bill

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Красноярск',
 'Екатеринбург',
 'Нижний Новгород',
 'Самара',
 'Казань',
 'Пермь',
 'Омск',
 'Челябинск',
 'Ростов-на-Дону',
 'Волгоград',
 'Уфа',
 'Воронеж',
 'Краснодар')

In [99]:
# текст запроса
query_5_7 = f'''select
           a.name,
           count(v.id)
           from
           areas a
           join
           vacancies v
           on a.id = v.area_id
           join
           employers e
           on e.id = v.employer_id
           where e.name = 'Яндекс'
           and a.name in {cities_bill}
           group by a.name
           union
           select
           'total',
           count(v.id)
           from
           areas a
           join
           vacancies v
           on a.id = v.area_id
           join
           employers e
           on e.id = v.employer_id
           where e.name = 'Яндекс'
           and a.name in {cities_bill}
           order by 2
    
'''

In [100]:
# результат запроса
df_5_7 = pd.read_sql_query(query_5_7, connection)
df_5_7

,name,count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

In [66]:
# выводы по анализу работодателей

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [163]:
# текст запроса
query_6_1 = f'''select
           count(id)
           from
           vacancies v
           where lower(name) similar to '%(data|данн)%'
'''

In [164]:
# результат запроса
df_6_1 = pd.read_sql_query(sa.text(query_6_1), connection)
df_6_1

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [247]:
ds_keyword = "'_*(data scientist|data science|исследователь данных|ml|machine learning|машинн%обучен%)_*'"
query_6_2 = f'''select
           count(id)
           from
           vacancies v
           where replace(lower(name), 'html', '') similar to {ds_keyword}
           and (lower(name) like '%junior%' or experience like 'Нет опыта' or employment like 'Стажировка')
           
'''

In [248]:
# результат запроса
df_6_2 = pd.read_sql_query(sa.text(query_6_2), connection)
df_6_2

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [249]:
# текст запроса
query_6_3 = f'''select
           count(id)
           from
           vacancies v
           where (replace(lower(name), 'html', '') similar to {ds_keyword})
           and 
           
'''

In [250]:
# результат запроса
df_6_3 = pd.read_sql_query(sa.text(query_6_3), connection)
df_6_3

,count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [251]:
# текст запроса
query_6_4 = f'''select
           count(id)
           from
           vacancies v
           where (replace(lower(name), 'html', '') similar to {ds_keyword})
           and (lower(key_skills) similar to '%(python)%')
'''

In [252]:
# результат запроса
df_6_4 = pd.read_sql_query(sa.text(query_6_4), connection)
df_6_4

,count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [253]:
# текст запроса
query_6_5 = f'''select
           round(avg(length(key_skills) - length(replace(key_skills, chr(9), ''))+1), 2)
           from
           vacancies v
           where (replace(lower(name), 'html', '') similar to {ds_keyword})
           and (lower(key_skills) similar to '%(python)%')
'''

In [254]:
# результат запроса
df_6_5 = pd.read_sql_query(sa.text(query_6_5), connection)
df_6_5

,round
0,6.59


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [259]:
# текст запроса
query_6_6 = f'''select
           experience,
           round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to)), 0)
           from
           vacancies v
           where (replace(lower(name), 'html', '') similar to {ds_keyword})
           and(salary_from is not null or salary_to is not null)
           group by experience
'''

In [260]:
# результат запроса
df_6_6 = pd.read_sql_query(sa.text(query_6_6), connection)
df_6_6

,experience,round
0,Более 6 лет,157933.0
1,Нет опыта,74643.0
2,От 1 года до 3 лет,144233.0
3,От 3 до 6 лет,256454.0


***

In [ ]:
# выводы по предметному анализу

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования